# Amazon Sagemaker 학습 스크립트

<p>이 예제는 LG에서 개발한 AI chip에서 동작할 수 있도록, Tensorflow 1.X, python2.7 버전에서 학습하기 위한 코드입니다. </p>
<p>이 코드는 <strong><a href="https://github.com/tensorflow/models/tree/master/research/slim" target="_blank" class ='btn-default'>TensorFlow-Slim image classification model library</a></strong>를 참고하여 Sagemaker에서 학습할 수 있는 실행 스크립트로 수정하여 작성하였습니다. Amazon SageMaker로 실행 스크립트를 구성하는 이유는 노트북의 스크립트에서 일부 파라미터 수정으로 동일 모델 아키텍처에 대해 hyperparamter가 변경된 다양한 모델을 원하는 형태의 다수 인프라에서 동시에 학습 수행이 가능하며, 가장 높은 성능의 모델을 노트북 스크립트 내 명령어로 바로 hosting 서비스가 가능한 Endpoint 생성을 할 수 있습니다.</p>

<p>이번 실습에서는 Amazon Sagemaker가 어떤 방식으로 학습이 되는지 설명되는 구조와 함께 학습하는 방법을 간단하게 체험해 보는 시간을 갖도록 하겠습니다.</p>

# 1. Sagemaker notebook 설명
<p>Sagemaker notebook은 완전 관리형 서비스로 컨테이너 기반으로 구성되어 있습니다. 사용자가 직접 컨테이너를 볼 수 없지만, 내부적으로는 아래와 같은 원리로 동작합니다. </p>
<p><img src="./fig/sm_notebook.png" width="700", height="70"></p>

- **S3 (Simple Storage Serivce)** : Object Storage로서 학습할 데이터 파일과 학습 결과인 model, checkpoint, tensorboard를 위한 event 파일, 로그 정보 등을 저장하는데 사용합니다.
- **SageMaker Notebook** : 학습을 위한 스크립트 작성과 디버깅, 그리고 실제 학습을 수행하기 위한 Python을 개발하기 위한 환경을 제공합니다.
- **Amazon Elastic Container Registry(ECR)** :  Docker 컨테이너 이미지를 손쉽게 저장, 관리 및 배포할 수 있게 해주는 완전관리형 Docker 컨테이너 레지스트리입니다. Sagemaker는 기본적인 컨테이너를 제공하기 때문에 별도 ECR에 컨테이너 이미지를 등록할 필요는 없습니다. 하지만, 별도의 학습 및 배포 환경이 필요한 경우 custom 컨테이너 이미지를 만들어서 ECR에 등록한 후 이 환경을 활용할 수 있습니다.

<p>학습과 추론을 하는 hosting 서비스는 각각 다른 컨테이너 환경에서 수행할 수 있으며, 쉽게 다량으로 컨테이너 환경을 확장할 수 있으므로 다량의 학습과 hosting이 동시에 가능합니다.   
</p>

# 2. 환경 설정

<p>Sagemaker 학습에 필요한 기본적인 package를 import 합니다. </p>
<p>boto3는 HTTP API 호출을 숨기는 편한 추상화 모델을 가지고 있고, Amazon EC2 인스턴스 및 S3 버켓과 같은 AWS 리소스와 동작하는 파이선 클래스를 제공합니다. </p>
<p>sagemaker python sdk는 Amazon SageMaker에서 기계 학습 모델을 교육 및 배포하기 위한 오픈 소스 라이브러리입니다.</p>

In [1]:
import sys

In [2]:
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install tensorflow_gpu==1.14

In [3]:
import os
import time
import sagemaker
import boto3
import tensorflow as tf
from PIL import Image

import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
from sagemaker.session import Session

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

%matplotlib inline

<p>SageMaker에서 앞으로 사용할 SageMaker Session 설정, Role 정보를 설정합니다. </p>

In [4]:
sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

sess = boto3.Session()
sm = sess.client('sagemaker')

## 3. S3의 저장 데이터 위치 가져오기
<p> 데이터를 정하기 위한 S3의 bucket 위치는 아래 data_bucket 이름으로 생성하며, 기본적으로 SageMaker에서 학습한 모델과 로그 정보를 남기는 위치는 자동으로 생성되는 bucket 이름으로 저장됩니다. </p>

In [5]:
# create a s3 bucket to hold data, note that your account might already created a bucket with the same name
account_id = sess.client('sts').get_caller_identity()["Account"]
data_bucket = 'sagemaker-experiments-{}-{}'.format(sess.region_name, account_id)
bucket = 'sagemaker-{}-{}'.format(sess.region_name, account_id)

try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=data_bucket)
    else:
        sess.client('s3').create_bucket(Bucket=data_bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print(e)

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


## 4. 이미지를 TFRecord 변경하기
<p>이미지 파일을 학습하기 위해 SageMaker Notebook 환경으로 upload를 합니다. 폴더 구조는 아래와 같은 형태로 구성되어야 합니다. </p>
<pre>
<div style='line-height:80%'>
    image_path/class1/Aimage_1<br>
                      Aimage_2<br>
                       ...<br>
                      Aimage_N<br>
    image_path/class2/Bimage_1<br>
                      Bimage_2<br>
                       ...<br>
                      Bimage_M<br>
</div>
</pre>
<p>생성된 TFRecord 파일은 아래 정의하신 dataset_dir에 저장이 됩니다. train과 test의 데이터 수는 향후 학습에서 활용하기 위해 train_num_data, test_num_data 변수에 저장합니다.</p>

In [6]:
sys.path.append('/home/ec2-user/SageMaker/src_dir/')

In [7]:
from datasets import image_to_tfrecord

In [8]:
dataset_dir = '/home/ec2-user/SageMaker/img_datasets'
image_path = '/home/ec2-user/SageMaker/data'

In [9]:
!rm -rf $dataset_dir

In [10]:
train_num_data, test_num_data = image_to_tfrecord.run(image_path, dataset_dir)







>> Reading file [['/home/ec2-user/SageMaker/data/dog/miniature_pinscher_199.jpg', '/home/ec2-user/SageMaker/data/dog/miniature_pinscher_157.jpg']] image 2/2geMaker/data/dog/saint_bernard_23.jpg', '/home/ec2-user/SageMaker/data/dog/beagle_49.jpg', '/home/ec2-user/SageMaker/data/dog/yorkshire_terrier_198.jpg', '/home/ec2-user/SageMaker/data/dog/german_shorthaired_112.jpg']] image 6/6
Finished converting the image dataset!


## 5. TFRecord를 S3에 upload 하기

<p>SageMaker 학습을 위해 TFRecord 파일을 S3에 upload합니다. TFRecord 은 이전에 지정한 data_bucket 내 prefix 하위 폴더에 저장됩니다. </p>

In [11]:
prefix = 'captured_data/tfrecord'
!aws s3 cp ./img_datasets s3://{data_bucket}/{prefix}/ --recursive

upload: img_datasets/captureddata_test.tfrecord to s3://sagemaker-experiments-us-east-2-322537213286/captured_data/tfrecord/captureddata_test.tfrecord
upload: img_datasets/labels.txt to s3://sagemaker-experiments-us-east-2-322537213286/captured_data/tfrecord/labels.txt
upload: img_datasets/captureddata_train.tfrecord to s3://sagemaker-experiments-us-east-2-322537213286/captured_data/tfrecord/captureddata_train.tfrecord


## 6. 학습 스크립트 코딩하기

<p>SageMaker에서 학습하는 것이 아니더라도 실제 모델 아키텍처와 학습을 위한 optimizer와 loss 함수 등을 정의하는 python 파일을 구성하게 됩니다. SageMaker에서 활용하는 python 파일도 동일한 python 파일을 사용하게 됩니다. 연계되는 다른 소스코드 파일이 있는 경우에도 별도 소스코드 수정 없이 학습이 가능하며, SageMaker에서 사용하기 위해서는 기존 python 파일에 SageMaker 학습에 사용할 수 있는 환경변수들만 추가하면 됩니다. 예를 들어, 환경변수 중 <code>SM_MODEL_DIR</code>은 컨테이너 환경에서는 <code>/opt/ml/model</code>를 의미합니다. 다양한 환경변수는 <strong><a href="https://github.com/aws/sagemaker-containers" target="_blank" class ='btn-default'>SageMaker Containers-IMPORTANT ENVIRONMENT VARIABLES</a></strong>를 참고하시기 바랍니다. </p><p>SageMaker 학습이 끝나면 자동은 컨테이너 환경은 삭제가 됩니다. 따라서, 학습이 완료된 모델 산출물과 다양한 output 파일은 S3로 저장해야 합니다. SageMaker는 자동으로 <code>SM_MODEL_DIR</code>에 저장된 최종 모델 파일을 학습이 끝난 다음 model.tar.gz로 압축하여 컨테이너 환경에서 S3의 특정 bucket에 저장하게 됩니다.</p><p> 별도 bucket을 설정하지 않으며, 기본적으로 생성되는 bucket에 저장됩니다. 이외 학습에 이용되는 python source code는 SageMaker 학습이 시작되면서 S3에 저장되며, 별도로 <code>SM_MODEL_DIR</code>에 checkpoint 또는 log 파일을 저장하게 되면 학습이 끝난 이후 자동으로 컨테이너 환경에서 S3로 저장된 파일들이 이동하게 됩니다. 이런 과정을 이해한다면 학습 시 저장되는 다양한 정보들을 저장한 다음 학습이 끝난 후 S3에서 download 받아 활용할 수 있습니다. </p>

<p>아래는 시간 관계 상 미리 작성한 python 학습 스크립트 코드 입니다.</p>

In [12]:
!pygmentize './src_dir/image_classifier.py'

# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Generic training script that trains a model using a given dataset."""

from __future__ import absolute_import, division, print_function

import argparse
import codecs
import glob
import io
import json
import logging
import math
import os
import re
import subprocess
import sys

import numpy as np
import PIL

imp

## 7. `TensorFlow` estimator를 이용한 training job 생성하기


<p><strong><code>sagemaker.tensorflow.TensorFlow</code></strong> estimator는 처음 실행하는 스크립트 위치와 다양한 연계 코드들이 위치한 디렉토리 정보를 찾아서 스크립트를 S3에 upload하고 SageMaker의 training job을 수행하게 됩니다. training job은 학습을 수행한 단위입니다. 학습을 1번 돌리면 training job이 1개 생성됩니다. 몇 가지 중요 파라미터를 아래와 같이 설명드립니다. </p>

- **entry_point** : 학습을 처음 실행하는 Python 소스 파일의 절대 또는 상대 경로이며, source_dir이 지정된 경우 entry_point는 source_dir 내 파일이 됩니다.
- **source_dir** : 학습에 연계되는 다양한 소스코드 파일이 들어 있는 디렉토리 위치이며, 절대, 상대 경로 또는 S3 URI가 모두 가능하며,source_dir이 S3 URI 인 경우 tar.gz 파일이 됩니다.
- **role** : Amazon SageMaker가 사용자를 대신해 작업(예: S3 버킷에서 모델 결과물이라고 하는 훈련 결과 읽기 및 Amazon S3에 훈련 결과 쓰기)을 수행하는 AWS Identity and Access Management(IAM) 역할입니다.
- **train_instance_count** : 학습을 수행하는 instance 개수를 정의할 수 있습니다.
- **train_instance_type** : 학습을 수행하는 instance 타입을 정의할 수 있습니다.
- **train_volume_size** : 학습 인스턴스에 연결할 Amazon Elastic Block Store(Amazon EBS) 스토리지 볼륨의 크기(GB)입니다. File 모드를 사용할 경우 이 값이 훈련 데이터를 충분히 저장할 수 있는 크기여야 합니다(File 모드가 기본값)
- **train_use_spot_instances** : 학습에서 SageMaker Managed Spot 인스턴스를 사용할지 여부를 지정합니다. 활성화되면 train_max_wait도 설정해야 합니다.
- **train_max_run** : 최대 학습 시간을 설정할 수 있으며, 이 시간이 지나면 Amazon SageMaker는 현재 상태에 관계없이 작업을 종료합니다. (기본값 : 24 * 60 * 60)
- **train_max_wait** : SageMaker Managed Spot 인스턴스를 기다리는 초 단위의 시간을 의미하는 것으로, 이 시간이 지나면 Amazon SageMaker는 스팟 인스턴스가 사용 가능해지기를 기다리는 것을 중지하며 결과는 fail이 됩니다.
- **framework_version** : 학습에 사용될 특정 Tensorflow 버전을 정의할 수 있습니다.
- **py_version** : 컨테이너 환경이 python3일 경우 py3, python2일 경우 py2로 설정하면 됩니다. python2는 지원이 중단되었지만 기존 python2로 구성된 파일들을 지원하기 위해 현재 계속 사용할 수 있습니다.
- **hyperparameters** : 학습에 사용할 하이퍼 파라미터를 정의할 수 있으며, 정의된 하이퍼 파라미터 값들은 모두 학습 컨테이너로 전송이 됩니다.

<p> 추가적으로 분산/ 멀티 GPU 학습도 가능합니다. SageMaker는 <strong><a href="https://github.com/horovod/horovod" target="_blank" class ='btn-default'>Horovod</a></strong>에 최적화된 환경을 제공하고 있으며, 자세한 내용은 <strong><a href="https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/tensorflow#distributed-training" target="_blank" class ='btn-default'>여기</a></strong>에서 확인이 가능합니다. 이번 학습에서는 분산/멀티 GPU 학습은 제외하였습니다.(단, 기존과 동일하게 python 소스코드에 분산/멀티 학습이 가능하도록 구성 필요) </p>


<p>S3에 저장된 TFRecord 파일의 위치를 다시 확인합니다.</p>

In [93]:
## Dataset 위치
inputs= 's3://{}/{}'.format(data_bucket, prefix)
inputs

's3://sagemaker-experiments-us-east-2-322537213286/captured_data/tfrecord'

In [94]:
hyperparameters = {
        'dataset_name' : 'captured_dataset',
        'model_name' : 'mobilenet_v1_025',
        'preprocessing_name' : 'mobilenet_v1',
        'image_size' : 128,
        'save_summaries_secs' : 300,
        'label_smoothing' : 0.1,
        'learning_rate_decay_factor' : 0.98,
        'num_epochs_per_decay' : 2.5,
        'moving_average_decay' : 0.9999,
        'batch_size' : 128,
        'max_number_of_steps' : 200,
        'eval_batch_size' : 1000,
        'train_num_data' : train_num_data,
        'test_num_data': test_num_data,
        'finetune_checkpoint_path' : 'fine_tune_checkpoint/mobilenet_v1_0.25_128.ckpt',
        'checkpoint_exclude_scopes' : 'MobilenetV1/Logits,MobilenetV1/AuxLogits',
    }

In [95]:
training_job_name = "{}-img-classifier-training-job".format(int(time.time()))
estimator = TensorFlow(entry_point='image_classifier.py',
                       source_dir='src_dir',
                       role=role,
                       train_instance_count=1,
                       train_instance_type='ml.p3.2xlarge',
                       train_use_spot_instances=True,  # spot instance 활용
                       train_volume_size=400,
                       train_max_run=12*60*60,
                       train_max_wait=12*60*60,
#                        train_instance_type='local_gpu',
                       framework_version='1.14.0',
                       py_version='py2',
                       hyperparameters=hyperparameters
                      )

## 8. Fit 함수로 학습 시작하기 

<p>학습을 시작하는 것은 <strong><code>estimator.fit (training_data_uri)</code></strong>이 호출되는 경우입니다. 여기에서 실제 데이터가 있는 S3의 위치가 입력으로 사용됩니다. <code>fit</code>에서는 <code>training</code>라는 기본 채널을 생성하며, 이 위치의 데이터는 S3에서 실제 컨테이너 환경에서는 <code>SM_CHANNEL_TRAINING</code> 위치로 복사되어 학습에 활용이 가능합니다. <code>fit</code>은 몇 가지 다른 유형의 입력도 허용하는데 자세한 내용은 <strong><a href="https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit" target="_blank" class ='btn-default'>API 문서</a></strong>를 참고하실 수 있습니다.</p>
<p> 학습이 시작되면 Tensorflow 컨테이너에서는 <code>image_classifier.py</code>를 실행되며, <code>Estimator</code>에서 <code>hyperparameters</code> 와 <code>model_dir</code>을 스크립트의 파라미터로 전달합니다. <code>model_dir</code>을 별도로 전달하지 않으며, 기본값은<strong>s3://[DEFAULT_BUCKET]/[TRAINING_JOB_NAME] </strong>이 되며 실제 스크립트 실행은 다음과 같습니다. </p>
    
```bash
python image_classifier.py --model_dir s3://[DEFAULT_BUCKET]/[TRAINING_JOB_NAME]
```
<p>학습이 완료되면 training job은 Tensorflow serving을 위해 saved model을 S3에 upload합니다.</p>
<p><code>fit</code>에서 <strong>wait=True</strong>로 설정할 경우 <strong>Synchronous</strong> 방식으로 동직하게 되며, <strong>wait=False</strong>일 경우 <strong>Aynchronous</strong> 방식으로 동작되어 여러 개의 Training job을 동시에 실행할 수 있습니다. </p>

In [96]:
estimator.fit(
    inputs = {'training': inputs},
    job_name=training_job_name,
    logs='All',
    wait=False
)
print("training_job_name : {}".format(training_job_name))

training_job_name : 1590924425-img-classifier-training-job


<p><strong>Aynchronous</strong>로 진행된 Training job은 아래와 같은 방법으로 진행상황을 실시간으로 확인할 수 있습니다.</p>

In [97]:
sm_sess = sagemaker.Session()
sm_sess.logs_for_job(estimator.latest_training_job.name, wait=True, log_type='All')

2020-05-31 11:27:13 Starting - Starting the training job...
2020-05-31 11:27:15 Starting - Launching requested ML instances.........
2020-05-31 11:28:45 Starting - Preparing the instances for training......
2020-05-31 11:29:59 Downloading - Downloading input data...
2020-05-31 11:30:28 Training - Downloading the training image...
2020-05-31 11:30:52 Training - Training image download completed. Training in progress.parser.parse_known_args() : (Namespace(adadelta_rho=0.95, adagrad_initial_accumulator_value=0.1, adam_beta1=0.9, adam_beta2=0.999, batch_size=128, checkpoint_exclude_scopes='MobilenetV1/Logits,MobilenetV1/AuxLogits', clone_on_cpu=False, current_host='algo-1', data_config={u'training': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, dataset_dir='/opt/ml/input/data/training', dataset_name='captured_dataset', end_learning_rate=0.01, eval_batch_size=1000, finetune_checkpoint_path='fine_tune_checkpoint/mobilenet_v1_0.25_

UnexpectedStatusException: Error for Training job 1590924425-img-classifier-training-job: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/bin/python image_classifier.py --batch_size 128 --checkpoint_exclude_scopes MobilenetV1/Logits,MobilenetV1/AuxLogits --dataset_name captured_dataset --eval_batch_size 1000 --finetune_checkpoint_path fine_tune_checkpoint/mobilenet_v1_0.25_128.ckpt --image_size 128 --label_smoothing 0.1 --learning_rate_decay_factor 0.98 --max_number_of_steps 200 --model_dir s3://sagemaker-us-east-2-322537213286/1590924425-img-classifier-training-job/model --model_name mobilenet_v1_025 --moving_average_decay 0.9999 --num_epochs_per_decay 2.5 --preprocessing_name mobilenet_v1 --save_summaries_secs 300 --test_num_data 2 --train_num_data 6 --use_grayscale False"

<p>학습이 모두 완료된 다음에 S3에서 모델 산출물을 SageMaker Notebook 환경으로 내려받습니다.</p>

In [80]:
artifacts_dir = estimator.model_dir.replace('model','')
print(artifacts_dir)
!aws s3 ls --human-readable {artifacts_dir}

s3://sagemaker-us-east-2-322537213286/1590923260-img-classifier-training-job/
                           PRE debug-output/
                           PRE output/
                           PRE source/


In [81]:
model_dir=artifacts_dir+'output/'
print(model_dir)
!aws s3 ls --human-readable {model_dir}

s3://sagemaker-us-east-2-322537213286/1590923260-img-classifier-training-job/output/
2020-05-31 11:11:55    2.8 MiB model.tar.gz


In [82]:
!rm -rf ./model_result/

In [83]:
import json , os

path = './model_result'
if not os.path.exists(path):
    os.makedirs(path)

!aws s3 cp {model_dir}model.tar.gz {path}/model.tar.gz
!tar -xzf {path}/model.tar.gz -C {path}

download: s3://sagemaker-us-east-2-322537213286/1590923260-img-classifier-training-job/output/model.tar.gz to model_result/model.tar.gz


<p>최종 결과물에는 tflite를 생성할 수 있도록 했습니다. 압축을 푼 다음 tflite 를 다시 활용하기 위해 S3에 파일을 upload 합니다.</p>

In [84]:
!aws s3 cp {path}/mobilenetv1_model.tflite {estimator.model_dir}/mobilenetv1_model.tflite

upload: model_result/mobilenetv1_model.tflite to s3://sagemaker-us-east-2-322537213286/1590923260-img-classifier-training-job/model/mobilenetv1_model.tflite



<p></p>
<p>Amazon SageMaker에서 모든 학습을 완료하였습니다. 이제 tflite를 이용하여 AI Chip에서 활용할 수 있도록 Convertor를 수행합니다. 이 작업은 Cloud9에서 수행합니다. </p>